# Age Transformation with GAN Demo

First, let's download the github repository and install all dependencies.

In [1]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

Cloning into 'Lifespan_Age_Transformation_Synthesis'...
remote: Enumerating objects: 618, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 618 (delta 4), reused 12 (delta 4), pack-reused 604
Receiving objects: 100% (618/618), 22.86 MiB | 12.31 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/Lifespan_Age_Transformation_Synthesis
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 16.0 MB/s 
     |████████████████████████████████| 235 kB 66.2 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
  Created wheel for visdom: filename=visdom-0.2.3-py3-none-any.whl size=1417107 sha256=f4951ca59325477601584c8600ec9e374ef91e5452b8138dc20fe93ac6158a8f
  Stored in directory: /root/.cache/pip/wheels/4d/73/32/0bbe55d2dccb9d80d3f020f474c15a5a1eef232817dcebe776
Successfully built visdom


Now let's download the pretrained models for males and females.

In [2]:
!python download_models.py

  0% 2.26k/213M [00:01<26:26:41, 2.24kB/s]Google Drive download failed.
Trying do download from alternate server

  0% 0.00/213M [00:00<?, ?B/s]
  0% 0.00/213M [00:00<?, ?B/s]
  0% 131k/213M [00:01<36:26, 97.4kB/s]
  0% 262k/213M [00:01<17:38, 201kB/s] 
  0% 524k/213M [00:01<07:58, 444kB/s]
  0% 918k/213M [00:01<04:14, 835kB/s]
  1% 1.97M/213M [00:02<01:42, 2.06MB/s]
  2% 3.80M/213M [00:02<00:49, 4.24MB/s]
  3% 6.82M/213M [00:02<00:26, 7.85MB/s]
  5% 9.96M/213M [00:02<00:18, 10.8MB/s]
  6% 13.1M/213M [00:02<00:15, 13.1MB/s]
  8% 16.3M/213M [00:02<00:13, 14.8MB/s]
  9% 19.4M/213M [00:02<00:12, 16.0MB/s]
 11% 22.5M/213M [00:03<00:11, 16.8MB/s]
 12% 25.6M/213M [00:03<00:10, 17.4MB/s]
 13% 28.7M/213M [00:03<00:10, 17.8MB/s]
 15% 31.9M/213M [00:03<00:09, 18.1MB/s]
 16% 35.0M/213M [00:03<00:09, 18.4MB/s]
 18% 38.1M/213M [00:03<00:09, 18.6MB/s]
 19% 40.1M/213M [00:04<00:10, 16.1MB/s]
 20% 41.8M/213M [00:04<00:11, 14.4MB/s]
 20% 43.4M/213M [00:04<00:13, 12.6MB/s]
 21% 45.1M/213M [00:04<00:13, 

Here, we import libraries and set options.

In [3]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

------------ Options -------------
activation: lrelu
batchSize: 1
checkpoints_dir: ./checkpoints
compare_to_trained_class: 1
compare_to_trained_outputs: False
conv_weight_norm: True
dataroot: ./datasets/males/
debug_mode: False
decoder_norm: pixel
deploy: False
display_id: 1
display_port: 8097
display_single_pane_ncols: 6
display_winsize: 256
fineSize: 256
full_progression: False
gen_dim_per_style: 50
gpu_ids: [0]
how_many: 50
id_enc_norm: pixel
image_path_file: None
in_the_wild: False
input_nc: 3
interp_step: 0.5
isTrain: False
loadSize: 256
make_video: False
max_dataset_size: inf
nThreads: 4
n_adaptive_blocks: 4
n_downsample: 2
name: debug
ngf: 64
no_cond_noise: False
no_flip: False
no_moving_avg: False
normalize_mlp: True
ntest: inf
output_nc: 3
phase: test
random_seed: -1
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
sort_classes: True
sort_order: ['0-2', '3-6', '7-9', '15-19', '30-39', '50-69']
trained_class_jump: 1
traverse: False
use_modulated_con

usage: ipykernel_launcher.py [-h] [--name NAME] [--gpu_ids GPU_IDS]
                             [--checkpoints_dir CHECKPOINTS_DIR]
                             [--batchSize BATCHSIZE] [--loadSize LOADSIZE]
                             [--fineSize FINESIZE] [--input_nc INPUT_NC]
                             [--output_nc OUTPUT_NC] [--dataroot DATAROOT]
                             [--sort_classes SORT_CLASSES]
                             [--sort_order SORT_ORDER]
                             [--resize_or_crop RESIZE_OR_CROP]
                             [--serial_batches] [--no_flip]
                             [--nThreads NTHREADS]
                             [--max_dataset_size MAX_DATASET_SIZE]
                             [--display_single_pane_ncols DISPLAY_SINGLE_PANE_NCOLS]
                             [--display_winsize DISPLAY_WINSIZE]
                             [--display_port DISPLAY_PORT]
                             [--display_id DISPLAY_ID]
                         

Next on, we call the data loader and the visualizer class that generates the video from the network outputs.

In [4]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


AgingDataLoader
dataset [MulticlassUnalignedDataset] was created


Here, we define our model.

NOTE: if you plan to try the method for a female, change opt.name to 'females_model'.

In [5]:
opt.name = 'males_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

Generator(
  (id_encoder): IdentityEncoder(
    (encoder): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): EqualConv2d(
        (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      )
      (2): PixelNorm()
      (3): ReLU(inplace=True)
      (4): ReflectionPad2d((1, 1, 1, 1))
      (5): EqualConv2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (6): PixelNorm()
      (7): ReLU(inplace=True)
      (8): ReflectionPad2d((1, 1, 1, 1))
      (9): EqualConv2d(
        (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
      )
      (10): PixelNorm()
      (11): ReLU(inplace=True)
      (12): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): EqualConv2d(
            (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          )
          (2): PixelNorm()
          (3): ReLU(inplace=True)
          (4): ReflectionPad2d((1, 1, 1, 1))
          (5): EqualConv2d(

InferenceModel(
  (netG): Generator(
    (id_encoder): IdentityEncoder(
      (encoder): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): EqualConv2d(
          (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
        )
        (2): PixelNorm()
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): EqualConv2d(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
        )
        (6): PixelNorm()
        (7): ReLU(inplace=True)
        (8): ReflectionPad2d((1, 1, 1, 1))
        (9): EqualConv2d(
          (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
        )
        (10): PixelNorm()
        (11): ReLU(inplace=True)
        (12): ResnetBlock(
          (conv_block): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): EqualConv2d(
              (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
            )
            (2): PixelNorm()
            (3): ReLU(inpl

OK, it's time to upload your image.

For best results, use images according to the following guidelines:

1. The image should contain a single face.
2. Image was taken from a digital camera (phone cameras are fine). Old images from film cameras would produce low quality results.
3. Pure RGB images only. No black & white, grayscale, sepia, or filtered images (e.g. Instagram filters).
4. Person's head should directly face the camera. Looking sideways/downwards/upwards degrades the results.
5. The person's face should not be occluded (or partially occluded) by any item.
6. Both eyes should be open and visible. (eyeglasses are ok, no sunglasses)

In [6]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Saving face2.PNG to face2.PNG
User uploaded file "face2.PNG"


Finally, we preprocess the image, run the network, and save the result.

In [7]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

Let's display at the results.

NOTE: if you're using chrome, uncomment the lines below. For some reason, mp4 files won't display on chrome browser, so we need to convert to webm.

In [8]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

# !pip3 install webm
# webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
# !webm -i $out_path $webm_out_path
# use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

You can download the result if you want

In [9]:
files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>